## Adam with Warm Up

`Adam with warmup`은 `Adam` 옵티마이저에 학습 초기에 lr을 서서히 올리는 '웜업(warmup)' 단계를 추가한 것이다. 

이 웜업 기간 동안 lr은 낮은 값에서 시작하여 점차 증가하며, 이는 모델이 더 안정적으로 학습을 시작하게 해 주어서 초기 학습의 불안정성을 줄이고 전체적인 학습 과정의 성능을 개선하는 데 도움을 준다. 특히, 큰 데이터셋이나 복잡한 모델에서 효과적이다.

In [2]:
from typing import Dict, Any, Tuple, Optional

from optimizers import WeightDecay, Adam

In [3]:
class AdamWarmup(Adam):
    def __init__(self, params, betas: Tuple[float, float] = (0.9, 0.999), 
                 eps: float = 1e-16, weight_decay: WeightDecay = WeightDecay(), 
                 optimized_update: bool = True, warmup: int = 0, 
                 min_lr: float = 1e-8, max_lr: float = 1e-3,
                 defaults: Dict[str, Any] | None = None):

        '''
        Initialize the Optimizer
            - params: the list of parameters
            - betas: tuple of (beta_1, beta_2)
            - lr: (maximum) learning rate alpha
            - eps: epsilon
            - weight_decay: instance of class WeightDecay
            - optimized_update: a flag whether to optimize the bias correction 
                                of the second moment by doing it after adding epsilon
            - warmup: number of warmup steps
            - min_lr: minimum learning rate to be the starting point for warmup phase
            - defaults: a dict of default for group values
        '''
        defaults = {} if defaults is None else defaults
        defaults.update({
            warmup : warmup,
            min_lr : min_lr,
            })
        
        super().__init__(params, betas, lr, eps, weight_decay, optimized_update, defaults)
    
    def get_lr(self, state: Dict[str, any], group: Dict[str, any]):
        # if current step is in warmup stage
        if group['warmup'] > state['step']:
            # linearly increasing lr from 0 to alpha
            return min_lr + state['step'] * group['lr'] / group['warmup']
        else:
            return group['lr']